In [1]:
from elasticsearch import Elasticsearch
import re
es = Elasticsearch()

In [2]:
syslog_count = es.count(index="homequitybank-2016.09.01", doc_type="syslog")['count']
syslog_count

441016

In [3]:
res = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=10, body={"query": {"match_all":{}}})
res

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVbjgTarh76HlwhVdgfF',
    '_index': 'homequitybank-2016.09.01',
    '_score': 1.0,
    '_source': {'@timestamp': '2016-09-01T02:08:30.721Z',
     '@version': '1',
     'customer': 'homequitybank',
     'host': '172.16.1.3',
     'message': '<85>1 2016-08-31T22:08:30--4:00 192.168.1.11 CP-GW - Log [Fields@1.3.6.1.4.1.2620 Action=" " UUid="{0x0,0x0,0x0,0x0}" log_id="4005" file_type="pif" scanned="0" scanned_last_day="0" scanned_last_week="0" scanned_last_month="0" malware_detected="0" malware_detected_last_day="0" malware_detected_last_week="0" malware_detected_last_month="0" threatcloud_scanned="0" threatcloud_scanned_last_day="0" threatcloud_scanned_last_week="0" threatcloud_scanned_last_month="0" threatcloud_malware="0" threatcloud_malware_detected_last_day="0" threatcloud_malware_detected_last_week="0" threatcloud_malware_detected_last_month="0" filter_by_static_analysis="0" filter_by_static_analysis

In [4]:
res1 = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=10, body={
  "query": { "bool" : { "must" : { "match": { "message":"product_family=\"Network\"" } } } } })
print(res1)

{'hits': {'hits': [{'_type': 'syslog', '_score': 0.27120787, '_source': {'customer': 'homequitybank', 'message': '<85>1 2016-08-31T22:40:56--4:00 192.168.1.11 CP-GW - Log [Fields@1.3.6.1.4.1.2620 Action=" " UUid="{0x57c79538,0x0,0xb01a8c0,0x749507b6}" roles="Anyuser" product="VPN-1 & FireWall-1" product_family="Network"]', '@timestamp': '2016-09-01T02:40:58.330Z', 'host': '172.16.1.3', 'type': 'syslog', '@version': '1'}, '_index': 'homequitybank-2016.09.01', '_id': 'AVbjnvADh76HlwhVdsSY'}], 'max_score': 0.27120787, 'total': 441006}, '_shards': {'failed': 0, 'successful': 5, 'total': 5}, 'timed_out': False, 'took': 41}


In [5]:
all_keys = set([])
for x in range(10):
    res = es.search(index="homequitybank-2016.09.01", doc_type="syslog",size=1, from_=x, body={
  "query": { "match": { "message":"src=" , "message":"dst"} } } )
    syslog = res['hits']['hits'][0]['_source']['message']
    kv = re.findall("(\w*=\".*?\")",syslog)
    sys_dict = {l[0]: l[1].strip("\"") for l in [l.split("=") for l in kv]}
    print(sys_dict)
    for key in sys_dict:
        all_keys.add(key)

{'UUid': '{0x57c81ed1,0x1d,0x30110ac,0xc0000000}', 'src': '172.16.5.96', 'Action': 'allow'}
{'UUid': '{0x0,0x0,0x0,0x0}', 'product': 'VPN-1 & FireWall-1', 'service': '500', 'src': '162.253.141.34', 'dst': '68.71.192.207', 'message_info': 'Address spoofing', 'product_family': 'Network', 's_port': '500', 'Action': 'monitor', 'proto': '17'}
{'UUid': '{0x0,0x0,0x0,0x0}', 'product': 'VPN-1 & FireWall-1', 'service': '23', 'src': '89.121.178.145', 'dst': '68.71.192.246', 'message_info': 'Address spoofing', 'product_family': 'Network', 's_port': '50554', 'Action': 'monitor', 'proto': '6'}
{'UUid': '{0x0,0x0,0x0,0x0}', 'product': 'VPN-1 & FireWall-1', 'service': '23', 'src': '123.14.222.251', 'dst': '68.71.192.239', 'message_info': 'Address spoofing', 'product_family': 'Network', 's_port': '35300', 'Action': 'monitor', 'proto': '6'}
{'UUid': '{0x0,0x0,0x0,0x0}', 'product': 'VPN-1 & FireWall-1', 'service': '137', 'src': '208.100.26.228', 'dst': '68.71.192.247', 'message_info': 'Address spoofing'

In [6]:
all_keys

{'Action',
 'UUid',
 'dst',
 'message_info',
 'product',
 'product_family',
 'proto',
 's_port',
 'service',
 'src'}